## 학습정리

### 24장. 데이터베이스와 SQL
* 데이터는 종종 데이터베이스(database)에 저장되어 제공
    * 데이터 베이스(database) : 데이터를 효과적으로 저장, 질의, 또는 쿼리(query)하기 위한 시스템
        * 관계형 데이터 베이스(relational database) : SQL로 쿼리하는 데이터 베이스(ex. PostgreSQL, MySQL, SQL Server)
        * SQL(structured query language) : 데이터를 테이블에 저장하고 조작할 수 있는 선언적(declarative)언어

####  24.1 CREATE TABLE, INSERT
* 관계형 데이터베이스 : 여러 테이블의 집합(과 그들간의 관계)
* 테이블 : 행의 집합.행렬과 크게 다르지 않음
    * 행렬과의 차이점 : 테이블에는 열의 이름과 데이터 타입(data type)이 명시된 스키마(schema)가 추가
    
####  24.2 UPDATE
* 데이터를 수정 또는 업데이트를 하기 위해 명시해야 하는 것 
    1. 어떤 테이블을 업데이트 할 것인가
    2. 어떤 행을 업데이트 할 것인가
    3. 어떤 필드를 업데이트 할 것인가
    4. 어떤 값으로 치환할 것인가 
    
####  24.3 DELETE
* 테이블의 행을 지우는 방법
    1. 테이블의 모든 행을 지움 : 위험
    2. 특정 조건을 만족하는 행만 WHERE절을 추가하여 제거 : 덜 위험
    
####  24.4 SELECT
* SELECT문을 이용해서 필요한 몇가지 행만 선택해서 보기 
    
####  24.5 GROUP BY
* GROUP BY : 지정된 열에서 동일한 값을 가진 행을 묶어서 MIN,MAX,COUNT
* WHERE절 : 병합을 하기 전에 행을 필터링
* HAVING절 : 병합된 결과를 필터링 

####  24.6 ORDER BY
* ORDER BY : 결과값을 정렬 
    * ASC : 오름차순
    * DESC : 내림차순

####  24.7 JOIN
* INNER JOIN : 매칭 조건에 맞는 조합만을 반환
* LEFT JOIN : 매칭 조건에 맞는 조합뿐 아니라 매칭되지 않은 왼쪽 테이블의 행까지 모두 반환
* RIGHT JOIN : 매칭되지 않은 오른쪽 테이블의 행을 보존
* FULL OUTER JOIN : 양쪽 테이블 모두 매칭되지 않는 행을 모두 보존 

####  24.8 서브 쿼리
* 서브 쿼리(subquery) : 쿼리의 결과물을 마치 또 다른 테이블인 것처럼 간주

####  24.9 인덱스
* 인덱스(index)의 사용
    * 특정 값을 가진 행을 찾을 때 테이블의 모든 행을 훑으면 시간의 소모가 큼
    * join 알고리즘도 비효율적
        * 전체 테이블을 훑어보지 않고도 매칭되는 행을 찾을 수 있게 됨 
    * 열에 제약 조건을 주고 싶을 때
        * 인덱스에 unique조건이 걸려 있다면, 중복된 값을 생성하려고 할 때 에러가 발생하도록 만들 수 있음

####  24.10 쿼리 최적화
* 필터링을 한 후 JOIN
    * 더 적은 수의 행에 JOIN을 하는 방법이 효율적 
* JOIN을 한 후 필터링
* SQL상에서는 쿼리 엔진이 알아서 판단하고 실행 
    
####  24.11 NoSQL
* NoSQL : 데이터베이스의 최근 트렌드 
    * 비관계형(nonrelational) 데이터 베이스
    * ex. MongoDB : 스키마와 행을 사용하지 않고 JSON 문서의 형태로 데이터를 표현하는 데이터베이스

* 컬럼형 데이터 베이스 : 행 대신 열에 데이터를 저장하는 데이터 베이스 
    
####  24.12 더 공부해 보고 싶다면
* 관계형 데이터 베이스를 간단히 사용해 보고 싶다면
    * SQLite
    * MySQL
    * PostgreSQL
* NoSQL
    * MongoDB
    * NoSQL 위키피디아 페이지 
    

## code

In [1]:
# users 데이터 셋 , user_id, name, num_friends
users = [[0, "Hero", 0],
         [1, "Dunn", 2],
         [2, "Sue", 3],
         [3, "Chi", 3]]


### SQL문
### 테이블 생성 

CREATE TABLE users (
    
    user_id INT NOT NULL,
    
    name VARCHAR(200),
    
    num_friends INT) ;

INSERT INTO users (user_id, name, num_friends) VALUES (0, 'Hero', 0);



In [15]:
from typing import Tuple, Sequence, List, Any, Callable, Dict, Iterator
from collections import defaultdict

# 이후에 사용할 타입 별칭
Row = Dict[str, Any]                        # 데이터 베이스 행
WhereClause = Callable[[Row], bool]         # 단일 행을 위한 술어
HavingClause = Callable[[List[Row]], bool]  # 여러 행에 대한 술어


In [80]:
# 생성자 

class Table :
    
    def __init__(self, columns: List[str], types: List[type]) -> None :
        assert len(columns) == len(types), "# of columns must == # of types"
        
        self.columns = columns # 열의 이름
        self.types = types # 열의 데이터 타입 
        self.rows : List[Row] = [] # 아직 데이터 없음
            
    # 열의 타입을 받기 위한 도우미 메서드 추가 
    def col2type(self, col: str) -> type :
        idx = self.columns.index(col) # 열의 인덱스를 찾고 
        return self.types[idx] # 해당 타입을 반환 
    
    # 추가 되는 값이 올바른지 확인해주는 insert 메서드 추가 
    # 맞는 개수의 값을 제공. 각각 올바른 타입이어야 함 
    def insert(self, values: list) -> None :
        # 값이 올바른 개수인지 확인
        if len(values) != len(self.types) :
            raise ValueError(f"You need to provide {len(self.types)} values")
            
        # 값의 타입이 맞는지 확인
        for value, typ3 in zip(values, self.types) :
            if not isinstance(value, typ3) and value is not None :
                raise TypeError(f"Expected type {typ3} but got {value}")
        
        # 대응되는 딕셔너리를 하나의 행으로 추가
        self.rows.append(dict(zip(self.columns, values)))
    
    def __getitem__(self, idx: int) -> Row:
        return self.rows[idx]
    
    def __iter__(self) -> Iterator[Row] :
        return iter(self.rows)
    
    def __len__(self) -> int :
        return len(self.rows)
    
    def __repr__(self) :
        """테이블을 예쁘게 표현하자: 열 이후에 행"""
        rows = "\n".join(str(row) for row in self.rows)
        
        return f"{self.columns}\n{rows}"
    
    # update함수 : 데이터 수정 및 업데이트 
    def update(self, updates: Dict[str, Any], predicate: WhereClause = lambda row: True) :
        # 일단 updates가 올바른 이름과 타입을 지녔는지 확인 
        for column, new_value in updates.items() :
            if column not in self.columns :
                raise ValueError(f"invalid column: {column}")
                
            typ3 = self.col2type(column)
            if not isinstance(new_value, typ3) and new_value is not None :
                raise TypeError(f"expected type {typ3}, but got {new_value}")
        
        # 이제 업데이트
        for row in self.rows :
            if predicate(row) :
                for column, new_value in updates.items() :
                    row[column] = new_value
                    
    # delete함수 
    def delete(self, predicate: WhereClause = lambda row: True) -> None :
        """predicate(술부)에 해당하는 모든 행을 제거"""
        self.rows = [row for row in self.rows if not predicate(row)]
    
    # select 함수
    def select(self, keep_columns : List[str] = None, additional_columns: Dict[str, Callable] = None) -> 'Table' :
        if keep_columns is None : # 만약 특정 열이 명시되지 않는다면 
            keep_columns = self.columns # 모든 열을 반환
        
        if additional_columns is None :
            additional_columns = {}
        
        # 새 열 이름과 타입
        new_columns = keep_columns + list(additional_columns.keys())
        keep_types = [self.col2type(col) for col in keep_columns]
        
        # 타입 어노테이션으로 부터 반환 타입을 알아 내는 방법은 다음과 같다.
        # 만약 calcualtion에 반환 타입이 없다면 코드가 멈춰버림 
        add_types = [calculation.__annotations__['return'] for calculation in additional_columns.values()]
        
        # 결과를 저장하기 위한 새로운 테이블 
        new_table = Table(new_columns, keep_types + add_types)
        
        for row in self.rows :
            new_row = [row[column] for column in keep_columns]
            for column_name, calculation in additional_columns.items() :
                new_row.append(calculation(row))
            new_table.insert(new_row)
        
        return new_table
    
    # where절 메서드
    def where(self, predicate : WhereClause = lambda row: True) -> 'Table' :
        """주어진 predicate에 해당하는 행만 반환"""
        where_table = Table(self.columns, self.types)
        for row in self.rows :
            if predicate(row) :
                values = [row[column] for column in self.columns]
                where_table.insert(values)
            return where_table
    
    # limit 메서드
    def limit(self, num_rows: int) -> 'Table' :
        """첫 num_rows만큼의 행만 반환"""
        limit_table = Table(self.columns, self.types)
        for i, row in enumerate(self.rows) :
            if i >= num_rows :
                break
            values = [row[column] for column in self.columns]
            limit_table.insert(values)
        return limit_table
    
    # GROUP BY
    def group_by(self, group_by_columns : List[str],
                 aggregates: Dict[str, Callable],
                 having: HavingClause = lambda group: True) -> 'Table' :
        
        grouped_rows = defaultdict(list)
        
        # 그룹을 채워 넣는다.
        for row in self.rows :
            key = tuple(row[column] for column in group_by_columns)
            grouped_rows[key].append(row)
        
        # 결과 테이블은 group_by열과 병합된 열들로 구성
        new_columns = group_by_columns + list(aggregates.keys())
        group_by_types = [self.col2type(col) for col in group_by_columns]
        aggregate_types = [agg.__annotations__['return']
                           for agg in aggregates.values()]
        result_table = Table(new_columns, group_by_types + aggregate_types)
        
        for key, rows in grouped_rows.items() :
            if having(rows) :
                new_row = list(key)
                for aggregate_name, aggregate_fn in aggregates.items() :
                    new_row.append(aggregate_fn(rows))
                result_table.insert(new_row)
            
            return result_table
    
    # ORDER BY
    def order_by(self, order: Callable[[Row], Any]) -> 'Table' :
        new_table = self.select() # 기존 테이블 복사
        new_table.rows.sort(key = order)
        return new_table
        
    
    def join(self, other_table: 'Table', left_join: bool = False) -> 'Table' :
        
        join_on_columns = [c for c in self.columns if c in other_table.columns] # 양쪽테이블이 모두 포함하는 열
        
        additional_columns = [c for c in other_table.columns if c not in join_on_columns] # 오른쪽 테이블에만 존재하는 열
        
        # 왼쪽 테이블의 모든 열 + 오른쪽 테이블의 추가적인 열 
        new_columns = self.columns + additional_columns
        new_types = self.types + [other_table.col2type(col) for col in additional_columns]
        
        join_table = Table(new_columns, new_types)
        
        for row in self.rows :
            def is_join(other_row) :
                return all(other_row[c] == row[c] for c in join_on_columns)
            
            other_rows = other_table.where(is_join).rows
            
            # 반환되는 테이블의 각 행을 현재 행과 일치하는 행으로 구성
            for other_row in other_rows :
                join_table.insert([row[c] for c in self.columns] + [other_row[c] for c in additional_columns])
                
            # 만약 일치하는 행이 없다면 left join을 의미하며 None을 포함하는 테이블을 반환
            if left_join and not other_rows :
                join_table.insert([row[c] for c in self.columns] + [None for c in additional_columns])
            
        return join_table

In [81]:
# User 테이블을 생성
users = Table(['user_id', 'name', 'num_friends'], [int, str, int]) 
users.insert([0, "Hero", 0]) 
users.insert([1, "Dunn", 2]) 
users.insert([2, "Sue", 3]) 
users.insert([3, "Chi", 3]) 
users.insert([4, "Thor", 3]) 
users.insert([5, "Clive", 2]) 
users.insert([6, "Hicks", 3]) 
users.insert([7, "Devin", 2]) 
users.insert([8, "Kate", 2]) 
users.insert([9, "Klein", 3]) 
users.insert([10, "Jen", 1])


In [31]:
users[1]['name']

'Dunn'

### SQL
UPDATE users 

SET num_friends = 3

WHERE user_id = 1;

In [41]:
# 기존값
users[1]['num_friends']

2

In [42]:
# update후 값
users.update({'num_friends' : 3}, lambda row : row['user_id'] == 1)
users[1]['num_friends']

3

### SQL
DELETE FROM users;

DELETE FROM users WHERE user_id = 1;

In [39]:
# 삭제 코드
users.delete(lambda row: row["user_id"] == 1) # user 1인 행을 삭제 
users.delete() # 모든 행을 삭제 

<bound method Table.delete of ['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}>

### SQL

SELECSELECT * FROM users;

SELECT * FROM users LIMIT 2;

SELECT user_id FROM users;

SELECT user_id FROM users WHERE name = 'Dunn';



In [82]:
# SELECSELECT * FROM users;
all_users = users.select()
print(len(all_users))

# SELECT * FROM users LIMIT 2;
two_users = users.limit(2)
print(len(two_users))

# SELECT user_id FROM users;
just_ids = users.select(keep_columns=['user_id'])
print(just_ids.columns)

# SELECT user_id FROM users WHERE name = 'Dunn';
dunn_ids = (users.where(lambda row : row["name"] == "Dunn").select(keep_columns=["user_id"]))
print(dunn_ids)

# SELECT LENGTH(name) AS name_length FROM users;
def name_length(row) -> int: return len(row["name"])

name_lengths = users.select(keep_columns = [], additional_columns = {"name_length" : name_length})
print(name_lengths[0]['name_length'])

11
2
['user_id']
['user_id']

4


### SQL
SELECT LENGTH(name) as name_length,
    
    MIN(user_id) AS min_user_id,
    
    COUNT(*) AS num_users
    
    FROM users
    
    GROUP BY LENGTH(name);
    

In [83]:
def min_user_id(rows) -> int :
    return min(row["user_id"] for row in rows)

def length(rows) -> int :
    return len(rows)
    
stats_by_length = (
    users
    .select(additional_columns = {"name_length" : name_length})
    .group_by(group_by_columns = ["name_length"], 
              aggregates = {"min_user_id" : min_user_id,
                            "num_users" : length})
)


In [84]:
def first_letter_of_name(row: Row) -> str :
    return row["name"][0] if row["name"] else ""

def average_num_friends(rows : List[Row]) -> float :
    return sum(row["num_friends"] for row in rows) / len(rows)

def enough_friends(rows : List[Row]) -> bool :
    return average_num_friends(rows) > 1

avg_friends_by_letter = (
    users
    .select(additional_columns={'first_letter' : first_letter_of_name})
    .group_by(group_by_columns=['first_letter'], 
              aggregates = {"avg_num_friends" : average_num_friends}, 
              having = enough_friends)
)

def sum_user_ids(rows: List[Row]) -> int :
    return sum(row["user_id"] for row in rows)

user_id_sum =(
    users
    .where(lambda row: row["user_id"] > 1)
    .group_by(group_by_columns = [], 
              aggregates = {"user_id_sum" : sum_user_ids })
)

### SQL

SELECT * FROM users

ORDER BY name

LIMIT 2;

In [85]:
friendliest_letters = (
    avg_friends_by_letter
    .order_by(lambda row: -row["avg_num_friends"])
    .limit(4))


In [86]:
friendliest_letters

['first_letter', 'avg_num_friends']
{'first_letter': 'H', 'avg_num_friends': 1.5}

### SQL

CREATE TABLE user_interests (
    
    user_id INT NOT NULL,
    
    interest VARCHAR(!00) NOT NULL

);

In [87]:
# 테이블 생성
user_interests = Table(['user_id', 'interest'], [int, str])
user_interests.insert([0, "SQL"])
user_interests.insert([0, "NoSQL"])
user_interests.insert([2, "SQL"])
user_interests.insert([2, "MySQL"])

### SQL
#### INNER JOIN : SQL에 관심 있는 사용자 목록을 얻고 싶다면 

SELECT users.name

FROM users

JOIN user_interests

ON users.user_id = user_interests.user_id

WHERE user_interests.interest = 'SQL'

#### LEFT JOIN

SELECT users.id, COUNT(user_interests.interest) AS num_interests

FROM users

LEFT JOIN user_interests

ON users.user_id = user


In [88]:
# SQL에 관심있는 사용자들을 찾기

sql_users = (
    users
    .join(user_interests)
    .where(lambda row: row["interest"] == "SQL")
    .select(keep_columns = ["name"])
)

In [89]:
sql_users

['name']
{'name': 'Hero'}

In [90]:
def count_interests(rows : List[Row]) -> int :
    """None이 아닌 관심사의 개수를 세어 준다"""
    return len([row for row in rows if row["interest"] is not None])

user_interest_count = (
    users
    .join(user_interests, left_join = True)
    .group_by(group_by_columns=["user_id"],
             aggregates={"num_interests" : count_interests})
)

### SQL
#### SUBQUERY 

SELECT MIN(user_id) AS min_user_id FROM

(SELECT user_id FROM user_interests WHERE interest = 'SQL') sql_interests;


In [92]:
# 서브쿼리 : 쿼리의 결과가 실제로 테이블
likes_sql_user_ids = (
    user_interests
    .where(lambda row: row["interest"] == 'SQL')
    .select(keep_columns=['user_id'])
)
likes_sql_user_ids.group_by(group_by_columns = [],
                            aggregates = {"min_user_id" : min_user_id })


['min_user_id']
{'min_user_id': 0}